# Trying Contextual Bandit Algorithms 

The two notebooks on bandits need to be finished and uploaded in a single zip file on this link https://nextcloud.univ-lille.fr/index.php/s/pK7qsbgqgeirbGk by **January 30th, 2023**. Please include all the files that allow to run your code and use Name_FirstName.zip as the name of the folder. 

In [4]:
import numpy as np
from matplotlib import pyplot as plt
import time

# Bandit specific functions and classes (same as last time) 
import Arms as arm
from StochasticBandit import * 
from BanditBaselines import * # you will need to add UCB alpha to your baselines 

from Experiments import * # all the previous functions to run experiments

# I) Defining a Linear Bandit Environement 

For simplicity, we will first experiment in a setting in which the feature $x_a \in \mathbb{R}^d$ associated to each arm $a$ is fixed in every rounds. That is, the set of arms available in round $t$ is  $$\mathcal X_t = \{x_1,\dots,x_K\} \subseteq \mathbb R^d$$ and it can still be identified with $\{1,\dots,K\}$. 

In this linear bandit model, when arm $A_t \in \{1,\dots,K\}$ is chosen in round $t$, a reward 
$$r_t = x_{A_t}^\top \theta_\star + \varepsilon_t$$
is generated, for some $\theta_\star \in \mathbb R^d$ (unknown to the agent). To generate data, we will further assume that the noise is Gaussian with zero mean and some standard deviation $\sigma^2$. 

The following function creates such a linear bandit model and notably depends on a matrix $X \in \mathbb{R}^{K \times d}$ where each row $a$ of $X$ is $x_a^\top$. 

In [5]:
def LinearBandit(X,theta,sigma):
    """X : matrix of features of dimension (K,d), 
    theta : regression vector of dimension (d,1), 
    sigma : stdev of Gaussian noise"""
    Arms = []
    (K,d)=np.shape(X)
    for k in range(K):
        mu = np.dot(X[k,:],theta)[0]
        arm = arms.Gaussian(mu,sigma)
        Arms.append(arm)
    return MAB(Arms)


**Pick a random linear bandit instance with normalized features (i.e. $\|x_a\|\leq 1$) and $\|\theta\|=1$.** 

Display the means and the best arm and settle for a "reasonnably interesting" problem with a gap that is not too large between the best and second-best arm. You may wish to save some interesting pairs (X,$\theta$) of features and regression parameters to work on later, for different dimensions / number of arms. 

In [6]:
K=10
d=5 

X = np.random.normal(0,1,(K,d))

for k in range(K):
    X[k,:]=X[k,:]/np.linalg.norm(X[k,:])

theta = np.random.random((d,1))
theta = theta / np.linalg.norm(theta)

sigma=0.5

linear_bandit = LinearBandit(X,theta,sigma)

#print("the means are",linear_bandit.means)
print("the best arm is",linear_bandit.bestarm)
print("gaps are",linear_bandit.means[linear_bandit.bestarm]-linear_bandit.means)

# save features and theta if you which to try this bandit problem again
#np.savetxt("X1.csv", X, delimiter=",")
#np.savetxt("theta1.csv", X, delimiter=",")

# load saved features and theta
#X = np.genfromtxt("X1.csv", delimiter=",")
#theta = np.genfromtxt("theta1.csv", delimiter=",")
#linear_bandit = LinearBandit(X,theta,sigma)

the best arm is 2
gaps are [1.3812589  0.10524724 0.         0.45548005 1.08854924 0.88448009
 0.4095742  1.04788709 0.94367084 1.79158172]


# II) Implementing least-squares and the greedy strategy

The function below gives the code for the Follow-the-Leader algorithm using least-square estimation to estimate $\theta_\star$ and the mean reward of each arm, i.e. the algorithm selecting in round $t+1$ arm  
$$A_{t+1} = \underset{a \in \{1,\dots,K\}}{\text{argmax}} \ x_a^\top \hat{\theta}_t^{\lambda},$$
where $\hat{\theta}_t^{\lambda}$ is the regularized least-square estimate, with regularization parameter $\lambda$.

**Complete the code with the online least-square update that is required.**

In [ ]:
class Greedy:
    """follow the leader using least-square estimation"""
    def __init__(self,X,reg=1):
        # the algorithms is fed with the known matrix of features (K,d) and the regularization parameter
        self.features = X
        (self.nbArms,self.dimension) = np.shape(X)
        self.reg = reg
        self.clear()

    def clear(self):
        # initialize the design matrix, its inverse, 
        # the vector containing the sum of r_s*x_s and the least squares estimate
        self.Design = self.reg*np.eye(self.dimension)
        self.DesignInv = (1/self.reg)*np.eye(self.dimension)
        self.Vector = np.zeros((self.dimension,1))
        self.thetaLS = np.zeros((self.dimension,1)) # regularized least-squares estimate
    
    def chooseArmToPlay(self):
        # compute the vector of estimated means  
        muhat = np.dot(self.features,self.thetaLS)
        # select the arm with largest estimated mean 
        return randmax(muhat)


    def receiveReward(self,arm,reward):
        x = self.features[arm,:].reshape((self.dimension,1))
        self.Design = self.Design + np.dot(x,np.transpose(x)) 
        self.Vector = self.Vector + reward*x
        # online update of the inverse of the design matrix
        self.DesignInv =  ## TO BE COMPLETED
        # update of the least squares estimate 
        self.thetaLS = ## TO BE COMPLETED

    def name(self):
        return "greedyLB"

**Experiment with the greedy strategy on one run. Is it finding the best arm?** 

Keep in mind that things can vary a lot between two different runs, don't hesitate to try multiple times.

In [ ]:
T = 1000
strategy1 = Greedy(X)

selections,rewards = OneBanditOneLearnerOneRun(linear_bandit, strategy1, T)
    
ViewSelections(selections)
print("the best arm is",linear_bandit.bestarm)


The linear bandit model that we created is a particular instance of a multi-armed bandit model in which each arm is Gaussian with variance $\sigma^2$. Therefore, we can also apply a standard UCB algorithm for this problem (that will not leverage the fact that all the means depend on the same regression parameter). With a well-chosen parameter for the UCB, this algorithm is guaranteed to have logarithmic regret on any linear bandit model (but the regret may be smaller for an algorithm leveraging the linear structure).

**Compare the regret (estimated over multiple runs) of the greedy algorithm implemented above with that of a UCB algorithm and that of the Follow The Leader algorithm implemented last time (i.e. the greedy algorithm that does *not* use the linear structure).**

For UCB, use the best theoretically-valid tuning and justify it.

In [ ]:
alpha=1000 # use a more reasonnable value for alpha of course ;)
Nexp=200
T=2000

strategy1 = Greedy(X)
strategy2 = FTL(K)
strategy3 = UCB(K,alpha)

RunExpes([strategy1, strategy2,strategy3],linear_bandit,Nexp,T,10,"off")

# III) Smarter algorithms

We discussed the following two algorithms in class: 

* LinUCB taking as an input a threshold function $\beta(t,\delta)$: 
$$A_{t+1}^{\text{LinUCB}} = \underset{a \in \{1,\dots,K\}}{\text{argmax}} \left[x_a^\top \hat{\theta}_t^{\lambda} + \beta(t,\delta) ||x_a||_{\left(B_t^{\lambda}\right)^{-1}}\right]$$
* Thompson Sampling, taking as an input a posterior inflation parameter $v$ 
$$A_{t+1}^{\text{LinTS}} = \underset{a \in \{1,\dots,K\}}{\text{argmax}} \ x_a^\top \tilde{\theta}_t$$
where $\tilde{\theta}_t$ is a sample from $\mathcal{N}\left(\hat{\theta}_t^{\lambda}, v^2 \left(B_t^{\lambda}\right)^{-1}\right)$.

**In terms of computational complexity, which algorithm is preferable?**

**Recall the parameters that need to be tuned for each method, and the tuning for which the two algorithms have regret guarantees.**


Implement one of these contextual algorithms. 

**Compare two versions of this algorithms: one with the theoretical tuning seen in class, and one with a (better) heuristic tuning of your choice. Include the non-contextual UCB algorithm in your comparison. Comments on the results.**